# Week9 Homework
## Working with Strings

The data frame contains notes taken by staff members. Your task is to clean it up to make it usable for analysis. 

### Tasks
* Create start date, end date columns and populate the values (If notes don't include month or day use 1) i.e: 2018 => 2018-01-01

* Create phone number column (If notes don't include phone numbers, use None)
* Create a column `duration` showing difference between start & end dates in years as a whole number

In [ ]:
import pandas as pd
import numpy as np
import re
from datetime import datetime as dt


In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/msaricaumbc/DS601_Fall21/main/Week09/data/hw_data.csv', index_col=0)

In [ ]:
df

Regular expression to extract different date patterns is refered from Stack Overflow. https://stackoverflow.com/questions/46064162/extracting-dates-that-are-in-different-formats-using-regex-and-sorting-them-pa

In [ ]:
#Creating temperory data frame and extracting the start date in it using different format regular expression 

df_temp = pd.DataFrame(columns = ['format_1', 'format_2','format_3','Start_date'])
df_temp['SD']= df.notes.str.extract(r'([sS].*[eE])')

df_temp['format_1']=df_temp.SD.str.extract(r'((?:\d{,2}\s)?(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*(?:-|\.|\s|,)\s?\d{,2}[a-z]*(?:-|,|\s)?\s?\d{2,4})')
# Converting extracted numbers into date
df_temp['Start_date'] = pd.to_datetime(df_temp['format_1'], errors='coerce')

df_temp['format_2'] = df_temp.SD.str.extract(r'((?:\d{2,4})(?:(?:\/|-)\d{1,2})?(?:(?:\/|-)\d{2,4}))')
df_temp['Start_date'] = pd.to_datetime(df_temp.Start_date.fillna(df_temp.format_2), errors='coerce')


df_temp['format_3'] = df_temp.SD.str.extract(r'((?:\d{1,2}(?:-|\/))?\d{2,4})')
#As there are dates written in mm-yy format need to specify their format conversion (ex: 04/20 to 2020/04/01) 
df_temp['format_3'] = pd.to_datetime(df_temp['format_3'], format='%m/%y', errors='coerce')
df_temp['Start_date'] = pd.to_datetime(df_temp.Start_date.fillna(df_temp.format_3), errors='coerce')

df_temp['format_4'] = df_temp.SD.str.extract(r'((?:\d{1,2}(?:-|\/))?\d{4})')
df['Start_date'] = pd.to_datetime(df_temp.Start_date.fillna(df_temp.format_4), errors='coerce')

#Shuffling the Column to verify no pattern is missed 
df_shuffled=df.sample(frac=1).reset_index(drop=True)
df_shuffled.head(5)




In [ ]:
# Simmilar method is used for end dates with few changes in regular expression pattern

df_temp['ED1'] = df.notes.str.extract(r'((?:[eE](?:[nN]|[dD])).*(?:\s|\d))')
df_temp['ED2'] = df_temp.ED1.str.extract(r'((?:[eE](?:[nN]|[dD])).*(?:[pP](?:(?:[nN]|\W))))')
df_temp['ED1']=df_temp.ED2.fillna(df_temp.ED1)


df_temp['format_1']=df_temp.ED1.str.extract(r'((?:\d{,2}\s)?(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*(?:-|\.|\s|,)\s?\d{,2}[a-z]*(?:-|,|\s)?\s?\d{2,4})')
df_temp['Start_date'] = pd.to_datetime(df_temp['format_1'], errors='coerce')

df_temp['format_2'] = df_temp.ED1.str.extract(r'((?:\d{2,4})(?:(?:\/|-)\d{1,2})?(?:(?:\/|-)\d{2,4}))')
df_temp['Start_date'] = pd.to_datetime(df_temp.Start_date.fillna(df_temp.format_2), errors='coerce')


df_temp['format_3'] = df_temp.ED1.str.extract(r'((?:\d{1,2}(?:-|\/))?\d{2,4})')
df_temp['format_3'] = pd.to_datetime(df_temp['format_3'], format='%m/%y', errors='coerce')
df_temp['Start_date'] = pd.to_datetime(df_temp.Start_date.fillna(df_temp.format_3), errors='coerce')

df_temp['format_4'] = df_temp.ED1.str.extract(r'((?:\d{1,2}(?:-|\/))?\d{4})')
df['End_date'] = pd.to_datetime(df_temp.Start_date.fillna(df_temp.format_4), errors='coerce')

pd.set_option('display.max_colwidth', None)
df_shuffled=df.sample(frac=1).reset_index(drop=True)
df_shuffled

In [ ]:
#Calculating the difference between start and end date in terms of year and converting it into integer

df['Duration_years'] = (df['End_date'] - df['Start_date'])
df['Duration_years'] = (df.Duration_years / np.timedelta64(1,'Y')).astype(int)
df


In [ ]:
#Extracting phone numbers from notes

df_temp['format_1'] = df.notes.str.extract(r'((?:[pP](?:(?:[nN]|\W|\s)(?:[nN]|\W|\d|\s))).*(?:\s|\W|\b))')
df_temp['format_2'] = df.notes.str.extract(r'([pP].*[sS](?:[dD]|[tT]))')
df_temp['format_2'] = df_temp.format_2.fillna(df_temp.format_1)

#Stripping extra characters from phone number column
df_temp['Phone_number']=df_temp.format_2.str.strip('Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec')
df_temp['Phone_number']=df_temp.Phone_number.str.strip('PpnN#:;/SstTdDeE-\s')

#If phone number is not included then add 'None'
df['Phone_number']= df_temp.Phone_number.fillna('None')


pd.set_option('display.max_colwidth', None)
df_shuffled=df.sample(frac=1).reset_index(drop=True)
df_shuffled.head(10)
